In [1]:
%pip install evaluate
%pip install bert_score

In [2]:
%pip install transformers datasets evaluate bert_score sacrebleu


In [3]:
import pandas as pd
from evaluate import load
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from seq2seq import create_transformers_train_data, train_transformer, decode_with_transformer
from sklearn.model_selection import train_test_split


TASK 1


In [4]:
data = pd.read_csv('test_en_parallel.txt', sep='\t', header=None)
data = data[[0, 1]]
data.columns = ['NEGATIVE', 'POSITIVE']
data = data[:999]
sentences_ne = data['NEGATIVE'].values.tolist()
sentences_pos = data['POSITIVE'].values.tolist()

In [5]:
model_name = 't5-small'
model2_name="google/flan-t5-small"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
tokenizer2 = AutoTokenizer.from_pretrained(model2_name)
model_flan = AutoModelForSeq2SeqLM.from_pretrained(model2_name)

In [8]:
sentences_ne[0]

'Style 1'

In [9]:
sentences_ne = sentences_ne[1:]
sentences_pos = sentences_pos[1:]

train_sentences_ne, test_sentences_ne, train_sentences_pos, test_sentences_pos = train_test_split(
    sentences_ne, sentences_pos, test_size=0.2, random_state=42
)


In [10]:
train_set = create_transformers_train_data(train_sentences_ne, train_sentences_pos, tokenizer)
train_set


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 798
})

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [12]:
data_collator2 = DataCollatorForSeq2Seq(tokenizer=tokenizer2, model=model_flan)
train_loader2 = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator2)

In [13]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [14]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 2.9984
Epoch 2/5, Loss: 2.2949
Epoch 3/5, Loss: 2.1860
Epoch 4/5, Loss: 2.0842
Epoch 5/5, Loss: 1.9209


In [15]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in test_sentences_ne
]



In [16]:
predicted_sentences

['I will definitely be returning to this place ever',
 'no one was helpful and friendly there',
 'his egg sandwiches are great and are',
 'he read her pressure, though.',
 'Always a good experience the owner and staff',
 'our experience with this business was positive.',
 'the owners were friendly and so were the staff',
 'the selection was great and they knew what they',
 'I like to be screwed over,',
 'i love their flavored coffee.',
 'great place to see a show as every',
 'i liked the service so i will',
 "It's located in a s",
 'oj and jeremy did ',
 'as far as pizza goes this ranks with the',
 'stopped by for soda after being at the hobby',
 "I've been here several times and every",
 'i purchased any of the pictures based',
 'absolutely ridiculous stay away from this place.',
 'they had a fountain machine on site',
 'it is consistently good and the staff is always',
 'is that a good thing or a',
 'this is a great place to throw back',
 'great selection of parts, and very reasonabl

In [17]:
reference_sentences = test_sentences_pos
reference_sentences


['I will be returning to this place frequently.',
 'they were very friendly and apologetic.',
 'His egg sandwiches smell good and I like the texture.',
 'He could really read her pressure, though.',
 'always a great experience there with the owner and the rest of the team.',
 'our experience with this business was positive.',
 'friendly owners and staff and delicious food!',
 'they have a fantastic selection and a very knowledgeable staff.',
 'I thought the prices were fair.',
 'i love their flavored coffee.',
 'fantastic place to see a show as every seat is a great seat!',
 'the service was great and would gladly go back.',
 "it's located in a great part of scottsdale and was really accommodating.",
 'oj and jeremy did a great job!',
 "as far as pizza goes this ranks with the best i've ever eaten.",
 'after the hobby shop i stopped in for a soda and enjoyed it',
 "i've been here several times and every time has been great.",
 'I opted to purchase the pictures based on quality.',
 "abs

In [18]:
bleu = load('bleu')

In [19]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.21771956560751116,
 'precisions': [0.5802997858672377,
  0.37885095753538717,
  0.2677322677322677,
  0.19825436408977556],
 'brevity_penalty': 0.6624245068663416,
 'length_ratio': 0.7082912032355915,
 'translation_length': 1401,
 'reference_length': 1978}

In [20]:
bertscore = load('bertscore')

In [21]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.8333533406257629,
  0.6724292039871216,
  0.7484880685806274,
  0.9526574611663818,
  0.7901486158370972,
  1.0,
  0.6946770548820496,
  0.6934983134269714,
  0.6865250468254089,
  1.0000001192092896,
  0.7676137089729309,
  0.6957439184188843,
  0.7127090096473694,
  0.845238208770752,
  0.9076128005981445,
  0.7546533942222595,
  0.8749282360076904,
  0.7049331665039062,
  0.7940878868103027,
  0.9301154613494873,
  0.8016200065612793,
  0.7276256084442139,
  0.8375948667526245,
  0.9016156792640686,
  0.8073585033416748,
  1.0,
  0.9054520130157471,
  0.8882458806037903,
  0.9142580032348633,
  0.8107101917266846,
  0.8065215349197388,
  0.4605671167373657,
  0.7000287175178528,
  0.818105936050415,
  0.9489970803260803,
  0.8729620575904846,
  0.6844294667243958,
  0.6185781359672546,
  0.7052466869354248,
  0.6786362528800964,
  0.7165798544883728,
  0.8833008408546448,
  0.8953120708465576,
  0.9036262035369873,
  0.5943655967712402,
  0.6411830186843872,
  0.807

In [22]:
model2 = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [23]:
optimizer2 = AdamW(model2.parameters(), lr=0.005)

In [24]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 3.2992
Epoch 2/5, Loss: 2.4980
Epoch 3/5, Loss: 2.1019
Epoch 4/5, Loss: 1.8604
Epoch 5/5, Loss: 1.5759


In [25]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model2)
    for s in test_sentences_ne
]



In [26]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.2010648245789238,
 'precisions': [0.55,
  0.3411290322580645,
  0.23461538461538461,
  0.16547619047619047],
 'brevity_penalty': 0.6882445104354916,
 'length_ratio': 0.7280080889787665,
 'translation_length': 1440,
 'reference_length': 1978}

In [27]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.5857367515563965,
  0.7042635679244995,
  0.872645378112793,
  0.7134796380996704,
  0.8033262491226196,
  0.9222343564033508,
  0.7359428405761719,
  0.6376194953918457,
  0.5377529263496399,
  0.9775192737579346,
  0.7676137089729309,
  0.6599993109703064,
  0.6934676170349121,
  0.8452383279800415,
  0.9076128005981445,
  0.7818058133125305,
  0.894199013710022,
  0.7558174729347229,
  0.7940878868103027,
  0.8369182348251343,
  0.7966402769088745,
  0.7276256084442139,
  0.8367031812667847,
  0.9226561188697815,
  0.9144651889801025,
  1.0,
  0.602357029914856,
  0.8882458806037903,
  0.8761659860610962,
  0.894963264465332,
  0.758543848991394,
  0.5087246298789978,
  0.7327446937561035,
  0.8435277938842773,
  0.6473470330238342,
  0.8610639572143555,
  0.6844295263290405,
  0.7500964403152466,
  0.7560526132583618,
  0.6146231889724731,
  0.7724090814590454,
  0.8035290837287903,
  0.8284590840339661,
  0.8924539089202881,
  0.5746841430664062,
  0.8025668859481

In [28]:
model3 = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [104]:
optimizer3 = AdamW(model3.parameters(), lr=2e-4)

In [105]:
train_transformer(model3, train_loader, optimizer3, 3)

Epoch 1/3, Loss: 2.1910
Epoch 2/3, Loss: 0.4231
Epoch 3/3, Loss: 0.3723


In [106]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model3)
    for s in test_sentences_ne
]



/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [107]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.037153817669490476,
 'precisions': [0.37516005121638923,
  0.2138157894736842,
  0.12681912681912683,
  0.08611111111111111],
 'brevity_penalty': 0.21596251065512154,
 'length_ratio': 0.3948432760364004,
 'translation_length': 781,
 'reference_length': 1978}

In [108]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.4046753942966461,
  0.6077587604522705,
  0.0,
  0.0,
  0.4251728653907776,
  0.41130608320236206,
  0.39372777938842773,
  0.28971150517463684,
  0.4920315742492676,
  0.9134680032730103,
  0.7386143207550049,
  0.6539598107337952,
  0.4740484058856964,
  0.8687828779220581,
  0.6156032681465149,
  0.9097535610198975,
  0.8816815614700317,
  0.7770863771438599,
  0.7715142369270325,
  0.5151910781860352,
  0.3491978645324707,
  0.48720431327819824,
  0.3707890510559082,
  0.5052880048751831,
  0.7026274800300598,
  0.0,
  0.3217388391494751,
  0.8051904439926147,
  0.6577889919281006,
  0.730281412601471,
  0.3110421299934387,
  0.5456731915473938,
  0.434773713350296,
  0.0,
  0.9489973187446594,
  0.5995355248451233,
  0.43456465005874634,
  0.35834941267967224,
  0.9533443450927734,
  0.9480626583099365,
  0.4728546738624573,
  0.0,
  0.31336510181427,
  0.9199643731117249,
  0.7861483097076416,
  0.5112417936325073,
  0.5844853520393372,
  0.6575352549552917,
  0.

In [109]:
optimizer_flan = AdamW(model_flan.parameters(), lr=0.005)

In [110]:
train_transformer(model_flan, train_loader2, optimizer, 5)

Epoch 1/5, Loss: 3.0118
Epoch 2/5, Loss: 2.9894
Epoch 3/5, Loss: 3.0444
Epoch 4/5, Loss: 3.0551
Epoch 5/5, Loss: 3.0319


In [111]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model_flan)
    for s in test_sentences_ne
]



/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [112]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.10865067745803647,
 'precisions': [0.4158200290275762,
  0.20797962648556875,
  0.12525458248472504,
  0.07341772151898734],
 'brevity_penalty': 0.6469969859707346,
 'length_ratio': 0.6966632962588474,
 'translation_length': 1378,
 'reference_length': 1978}

In [113]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.7968602776527405,
  0.6400213241577148,
  0.7479581236839294,
  0.6526873111724854,
  0.6524489521980286,
  0.6748664379119873,
  0.6413506865501404,
  0.6110754013061523,
  0.5664645433425903,
  0.9022269248962402,
  0.3954220414161682,
  0.7551785111427307,
  0.7127090096473694,
  0.845238208770752,
  0.7226361036300659,
  0.6700975894927979,
  0.8749282360076904,
  0.785513162612915,
  0.7960894107818604,
  0.328110933303833,
  0.5216524600982666,
  0.5370437502861023,
  0.48006707429885864,
  0.7095051407814026,
  0.7375215291976929,
  0.8177042007446289,
  0.9443621635437012,
  0.8165127038955688,
  0.7075468301773071,
  0.8002712726593018,
  0.6791220307350159,
  0.48848849534988403,
  0.5893884301185608,
  0.8656299114227295,
  0.43515604734420776,
  0.8610639572143555,
  0.6582275629043579,
  0.6651477217674255,
  0.5366647243499756,
  0.5878394842147827,
  0.7143158316612244,
  0.6897064447402954,
  0.8092515468597412,
  0.9199645519256592,
  0.593216300010681

# TASK 2



In [37]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [38]:
instruction = 'Convert the negative-sentiment sentence into a positive-sentiment sentence: '

In [39]:
train_sentences_ne_with_ins = [f'{instruction}{s}' for s in train_sentences_ne]
test_sentences_ne_with_ins = [f'{instruction}{s}' for s in test_sentences_ne]


In [40]:
train_sentences_ne_with_ins


['Convert the negative-sentiment sentence into a positive-sentiment sentence: plus my sparkling wine flights were gross!',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: this is an old worn out hotel.',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: it was so odd, i asked her if she was joking?',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: terrible selection of games to play.',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: ra sushi, you are so blah to me.',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: hate their food and their passion.',
 "Convert the negative-sentiment sentence into a positive-sentiment sentence: you'll have zero appetite after the first bite.",
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: so sad but i am not going back any more.',
 'Convert the negative-sentiment sentence into

In [41]:
train_set = create_transformers_train_data(train_sentences_ne_with_ins, train_sentences_pos, tokenizer)
train_set


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 798
})

In [42]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [43]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [44]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 5.3931
Epoch 2/5, Loss: 4.0326
Epoch 3/5, Loss: 3.8591
Epoch 4/5, Loss: 3.7062
Epoch 5/5, Loss: 3.6770


In [45]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in test_sentences_ne_with_ins
]



In [46]:
predicted_sentences

['the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'they are so friendly and helpful.',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food is great and the food is great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great',
 'the food was great and the food was great'

In [47]:
bleu = load('bleu')
bertscore = load('bertscore')

In [48]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.0,
 'precisions': [0.15979672501411632,
  0.01336728198599618,
  0.0014587892049598833,
  0.0],
 'brevity_penalty': 0.8896891767303647,
 'length_ratio': 0.8953488372093024,
 'translation_length': 1771,
 'reference_length': 1978}

In [49]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.3201615810394287,
  0.38544151186943054,
  0.40252771973609924,
  0.2896732985973358,
  0.3703397512435913,
  0.4237363338470459,
  0.41614025831222534,
  0.3773347735404968,
  0.42609477043151855,
  0.6580000519752502,
  0.4238054156303406,
  0.4844617247581482,
  0.4097892642021179,
  0.4121224284172058,
  0.36749038100242615,
  0.3414917290210724,
  0.3970499038696289,
  0.35483282804489136,
  0.3840808868408203,
  0.3118559718132019,
  0.3953627943992615,
  0.414539635181427,
  0.3493325412273407,
  0.34511080384254456,
  0.3554987609386444,
  0.3587111830711365,
  0.34245020151138306,
  0.3658667206764221,
  0.3947199583053589,
  0.3192836046218872,
  0.39436841011047363,
  0.30599355697631836,
  0.409029483795166,
  0.3245847523212433,
  0.4085138738155365,
  0.32467663288116455,
  0.38025498390197754,
  0.4673857092857361,
  0.454133003950119,
  0.329084187746048,
  0.3982763886451721,
  0.60006183385849,
  0.37840449810028076,
  0.3257501721382141,
  0.38208431

In [50]:
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [114]:
optimizer2 = AdamW(model2.parameters(), lr=0.005)

In [115]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 1.4128
Epoch 2/5, Loss: 0.3641
Epoch 3/5, Loss: 0.3695
Epoch 4/5, Loss: 0.3584
Epoch 5/5, Loss: 0.3506


In [116]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in test_sentences_ne_with_ins
]



In [147]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.0037184926137839775,
 'precisions': [0.1171875,
  0.0072992700729927005,
  0.002506265664160401,
  0.0015698587127158557],
 'brevity_penalty': 0.4882079245923248,
 'length_ratio': 0.5824064711830131,
 'translation_length': 1152,
 'reference_length': 1978}

In [118]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.2981612980365753,
  0.3178398907184601,
  0.31050118803977966,
  0.33810317516326904,
  0.3632237911224365,
  0.4113060534000397,
  0.35150787234306335,
  0.32148048281669617,
  0.30374258756637573,
  0.3121767044067383,
  0.33831241726875305,
  0.3050498962402344,
  0.3418079614639282,
  0.3239739239215851,
  0.30143946409225464,
  0.3191652297973633,
  0.3194185197353363,
  0.316924124956131,
  0.3774178624153137,
  0.32501065731048584,
  0.2984519600868225,
  0.3970054090023041,
  0.3379361927509308,
  0.33904263377189636,
  0.32968804240226746,
  0.31141915917396545,
  0.31964847445487976,
  0.2971497178077698,
  0.3324042856693268,
  0.372806191444397,
  0.3200056254863739,
  0.29413682222366333,
  0.2792440354824066,
  0.31379610300064087,
  0.3005853295326233,
  0.33546921610832214,
  0.3157828748226166,
  0.33681419491767883,
  0.3555419445037842,
  0.3592424988746643,
  0.3111230134963989,
  0.3184296488761902,
  0.3232016861438751,
  0.2950442135334015,
  0.3

In [119]:
model3 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [120]:
optimizer3 = AdamW(model3.parameters(), lr=0.005)

In [121]:
train_transformer(model3, train_loader, optimizer3, 10)

Epoch 1/10, Loss: 2.1848
Epoch 2/10, Loss: 0.3667
Epoch 3/10, Loss: 0.3589
Epoch 4/10, Loss: 0.3593
Epoch 5/10, Loss: 0.3651
Epoch 6/10, Loss: 0.3566
Epoch 7/10, Loss: 0.3510
Epoch 8/10, Loss: 0.3531
Epoch 9/10, Loss: 0.3559
Epoch 10/10, Loss: 0.3484


In [122]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model3)
    for s in test_sentences_ne_with_ins
]



In [123]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.0,
 'precisions': [0.01, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.0001377596579815859,
 'length_ratio': 0.10111223458038422,
 'translation_length': 200,
 'reference_length': 1978}

In [124]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.3041570484638214,
  0.3678250014781952,
  0.3091214895248413,
  0.29347506165504456,
  0.3368609845638275,
  0.5771906971931458,
  0.4085646867752075,
  0.2871669828891754,
  0.32414817810058594,
  0.3236945569515228,
  0.3249615430831909,
  0.3187505602836609,
  0.32970961928367615,
  0.2980130612850189,
  0.2831638753414154,
  0.2809714376926422,
  0.33910608291625977,
  0.316924124956131,
  0.418375164270401,
  0.3001752495765686,
  0.35832351446151733,
  0.3348682224750519,
  0.3391930162906647,
  0.38183584809303284,
  0.31462153792381287,
  0.34154435992240906,
  0.3118501305580139,
  0.32460394501686096,
  0.3214654326438904,
  0.4213148057460785,
  0.31363776326179504,
  0.28051748871803284,
  0.2710123360157013,
  0.3401227295398712,
  0.3072599172592163,
  0.3413965404033661,
  0.3157828748226166,
  0.3534042537212372,
  0.31857410073280334,
  0.3592424988746643,
  0.35859745740890503,
  0.28639116883277893,
  0.31900733709335327,
  0.3130163252353668,
  0.34

TASK 3


In [61]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [62]:
bleu = load('bleu')
bertscore = load('bertscore')

In [63]:
instruction_A = "Rewrite the sentence to have a positive sentiment: "

In [64]:
train_sentences_ne_with_ins_a = [f'{instruction_A}{s}' for s in train_sentences_ne]
test_sentences_ne_with_ins_a = [f'{instruction_A}{s}' for s in test_sentences_ne]


In [65]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [66]:
train_set = create_transformers_train_data(train_sentences_ne_with_ins_a, train_sentences_pos, tokenizer)
train_set


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 798
})

In [67]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [68]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [69]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 5.0686
Epoch 2/5, Loss: 4.0026
Epoch 3/5, Loss: 3.8617
Epoch 4/5, Loss: 3.6810
Epoch 5/5, Loss: 3.5202


In [70]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in test_sentences_ne_with_ins_a
]


In [71]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.010560477001984647,
 'precisions': [0.15371716042481834,
  0.016991818753933293,
  0.004319654427645789,
  0.001682085786375105],
 'brevity_penalty': 0.8997434497981472,
 'length_ratio': 0.9044489383215369,
 'translation_length': 1789,
 'reference_length': 1978}

In [72]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.3201615810394287,
  0.38544151186943054,
  0.40252771973609924,
  0.2896732985973358,
  0.3703397512435913,
  0.4237362742424011,
  0.41614025831222534,
  0.3773347735404968,
  0.42609477043151855,
  0.3528033196926117,
  0.4238054156303406,
  0.5253618359565735,
  0.4143037796020508,
  0.41212236881256104,
  0.36749038100242615,
  0.3414917290210724,
  0.3970499038696289,
  0.35483282804489136,
  0.3840808868408203,
  0.3118559718132019,
  0.3953627943992615,
  0.414539635181427,
  0.3493325412273407,
  0.34511080384254456,
  0.351043164730072,
  0.3587111830711365,
  0.34245020151138306,
  0.3658667206764221,
  0.3947199583053589,
  0.3192836046218872,
  0.3968212604522705,
  0.30599355697631836,
  0.409029483795166,
  0.3245847523212433,
  0.4085138738155365,
  0.32467663288116455,
  0.38025498390197754,
  0.4673857092857361,
  0.454133003950119,
  0.329084187746048,
  0.3982763886451721,
  0.33045873045921326,
  0.37840449810028076,
  0.3257501721382141,
  0.382084

In [125]:
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [126]:
optimizer2 = AdamW(model2.parameters(), lr=0.005)

In [127]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 1.3549
Epoch 2/5, Loss: 0.3634
Epoch 3/5, Loss: 0.3511
Epoch 4/5, Loss: 0.3548
Epoch 5/5, Loss: 0.3529


In [128]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model2)
    for s in test_sentences_ne_with_ins_a
]


In [129]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.0,
 'precisions': [0.01, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.0001377596579815859,
 'length_ratio': 0.10111223458038422,
 'translation_length': 200,
 'reference_length': 1978}

In [130]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.30415716767311096,
  0.3678250014781952,
  0.30912166833877563,
  0.29347512125968933,
  0.3368609845638275,
  0.577190637588501,
  0.4085647463798523,
  0.2871670424938202,
  0.3241482973098755,
  0.32369470596313477,
  0.3249616026878357,
  0.3187505900859833,
  0.32970964908599854,
  0.2980131208896637,
  0.28316396474838257,
  0.28097161650657654,
  0.33910611271858215,
  0.315398633480072,
  0.4183752238750458,
  0.30017533898353577,
  0.35832348465919495,
  0.39700525999069214,
  0.33919310569763184,
  0.3818359673023224,
  0.31462159752845764,
  0.3415444493293762,
  0.3118501901626587,
  0.32460397481918335,
  0.32146555185317993,
  0.4213148355484009,
  0.313637912273407,
  0.280517578125,
  0.2710123658180237,
  0.3401227593421936,
  0.30726000666618347,
  0.3413967192173004,
  0.2976476550102234,
  0.3534042239189148,
  0.3185741901397705,
  0.41312605142593384,
  0.35859742760658264,
  0.2863912582397461,
  0.31900742650032043,
  0.3130163252353668,
  0.348

In [131]:
model3 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [132]:
optimizer3 = AdamW(model3.parameters(), lr=0.005)

In [133]:
train_transformer(model3, train_loader, optimizer, 10)

Epoch 1/10, Loss: 6.3400
Epoch 2/10, Loss: 6.3487
Epoch 3/10, Loss: 6.3004
Epoch 4/10, Loss: 6.3292
Epoch 5/10, Loss: 6.3498
Epoch 6/10, Loss: 6.3338
Epoch 7/10, Loss: 6.2977
Epoch 8/10, Loss: 6.3755
Epoch 9/10, Loss: 6.3548
Epoch 10/10, Loss: 6.3569


In [134]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model3)
    for s in test_sentences_ne_with_ins_a
]


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [135]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.0037184926137839775,
 'precisions': [0.1171875,
  0.0072992700729927005,
  0.002506265664160401,
  0.0015698587127158557],
 'brevity_penalty': 0.4882079245923248,
 'length_ratio': 0.5824064711830131,
 'translation_length': 1152,
 'reference_length': 1978}

In [136]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.5820977687835693,
  0.3178398907184601,
  0.5726536512374878,
  0.547299325466156,
  0.5749608874320984,
  0.6180938482284546,
  0.5058335065841675,
  0.32148057222366333,
  0.5741048455238342,
  0.5500986576080322,
  0.5755096673965454,
  0.6449031829833984,
  0.5719521641731262,
  0.5375142097473145,
  0.5478127002716064,
  0.5814505815505981,
  0.5654788613319397,
  0.630186915397644,
  0.555109441280365,
  0.5969512462615967,
  0.5815485119819641,
  0.6396365165710449,
  0.3296988606452942,
  0.5698248147964478,
  0.5511508584022522,
  0.5250144600868225,
  0.5127730369567871,
  0.5575441122055054,
  0.3229198455810547,
  0.5945433378219604,
  0.5669516324996948,
  0.5610246658325195,
  0.2650868892669678,
  0.5761563777923584,
  0.5829634070396423,
  0.5543553233146667,
  0.5301915407180786,
  0.5948017835617065,
  0.5990333557128906,
  0.5699320435523987,
  0.5686416625976562,
  0.5663043260574341,
  0.5686885118484497,
  0.5890114307403564,
  0.5247012376785278,

Sentiment


In [84]:
train_data = []
for s in train_sentences_pos:
    train_data.append((s, "positive"))
for s in train_sentences_ne:
    train_data.append((s, "negative"))

test_data = []
for s in test_sentences_pos:
    test_data.append((s, "positive"))
for s in test_sentences_ne:
    test_data.append((s, "negative"))

sentences = [sentence for sentence, _ in train_data]
labels = [label for _, label in train_data]
test_sentences = [sentence for sentence, _ in test_data]
test_labels = [label for _, label in test_data]
prompt = (
    "Classify sentiment (positive or negative).\n"
    "Sentence: I hated the service.\n"
    "Sentiment: negative\n\n"
    "Sentence: "
)
inputs = [prompt + s for s in sentences]
test_inputs = [prompt + s for s in test_sentences]
model_name="t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [85]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [86]:
train_set = create_transformers_train_data(inputs, labels, tokenizer)
train_set

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1596
})

In [87]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [88]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [89]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 1.3655
Epoch 2/5, Loss: 0.3619
Epoch 3/5, Loss: 0.3654
Epoch 4/5, Loss: 0.3621
Epoch 5/5, Loss: 0.3523


In [90]:
predictions = [
    decode_with_transformer(s, tokenizer, model)
    for s in test_inputs
]



In [91]:
predictions = [p.lower().strip() for p in predictions]
predictions

['negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',

In [92]:
correct = 0
total = len(test_labels)

for p, r in zip(predictions, test_labels):
    if p == r:
        correct += 1

accuracy = correct / total
accuracy


0.5475

In [93]:
set(predictions)


{'negative', 'positive'}

In [137]:
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [138]:
optimizer2 = AdamW(model2.parameters(), lr=0.005)

In [139]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 1.4130
Epoch 2/5, Loss: 0.3625
Epoch 3/5, Loss: 0.3588
Epoch 4/5, Loss: 0.3553
Epoch 5/5, Loss: 0.3556


In [140]:
predictions = [
    decode_with_transformer(s, tokenizer, model2)
    for s in test_inputs
]



In [141]:
correct = 0
total = len(test_labels)

for p, r in zip(predictions, test_labels):
    if p == r:
        correct += 1

accuracy = correct / total
accuracy


0.5

In [142]:
model3 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [143]:
optimizer3 = AdamW(model3.parameters(), lr=0.005)

In [144]:
train_transformer(model3, train_loader, optimizer3, 10)

Epoch 1/10, Loss: 1.2609
Epoch 2/10, Loss: 0.3615
Epoch 3/10, Loss: 0.3565
Epoch 4/10, Loss: 0.3659
Epoch 5/10, Loss: 0.3560
Epoch 6/10, Loss: 0.3571
Epoch 7/10, Loss: 0.3582
Epoch 8/10, Loss: 0.3515
Epoch 9/10, Loss: 0.3501
Epoch 10/10, Loss: 0.3496


In [145]:
predictions = [
    decode_with_transformer(s, tokenizer, model3)
    for s in test_inputs
]



In [146]:
correct = 0
total = len(test_labels)

for p, r in zip(predictions, test_labels):
    if p == r:
        correct += 1

accuracy = correct / total
accuracy


0.5